# EEG Challenge 2025 - S4 Model MVP Solution
## Challenge 1: Cross-Task Transfer Learning with S4 Architecture

This notebook implements a minimal viable product (MVP) solution for the EEG Foundation Challenge 2025 using the S4 (Structured State Space) model architecture for cross-task transfer learning.

**Goal**: Predict response times from EEG signals using knowledge transfer from passive to active tasks.

**Key Components**:
- S4 model for temporal EEG sequence modeling
- Cross-paradigm transfer (SuS → CCD)
- Response time regression from 129-channel EEG data

In [ ]:
# Import required libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# Check for GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Install required packages (uncomment if needed)
# !pip install braindecode eegdash mne

## 1. S4 Model Architecture

We implement a simplified S4 model based on the existing S4 components from the eeg-pretraining directory, adapted for the challenge requirements.

In [ ]:
# Simplified S4 Layer Implementation
class S4Layer(nn.Module):
    """Simplified S4 layer for EEG sequence modeling"""
    
    def __init__(self, d_model, d_state=64, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        
        # Simplified state space parameters
        self.A = nn.Parameter(torch.randn(d_state, d_state) * 0.1)
        self.B = nn.Parameter(torch.randn(d_state, d_model) * 0.1)
        self.C = nn.Parameter(torch.randn(d_model, d_state) * 0.1)
        self.D = nn.Parameter(torch.randn(d_model) * 0.1)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        """Forward pass through S4 layer
        Args:
            x: Input tensor of shape (batch, seq_len, d_model)
        Returns:
            Output tensor of shape (batch, seq_len, d_model)
        """
        batch_size, seq_len, d_model = x.shape
        
        # Initialize state
        h = torch.zeros(batch_size, self.d_state, device=x.device)
        outputs = []
        
        # Process sequence step by step
        for t in range(seq_len):
            u = x[:, t, :]  # Input at time t
            
            # State space update: h_{t+1} = A*h_t + B*u_t
            h_next = torch.matmul(h, self.A.T) + torch.matmul(u, self.B.T)
            
            # Output: y_t = C*h_t + D*u_t
            y = torch.matmul(h, self.C.T) + u * self.D
            
            outputs.append(y)
            h = h_next
        
        output = torch.stack(outputs, dim=1)  # (batch, seq_len, d_model)
        return self.dropout(output)

In [ ]:
class EEGS4Model(nn.Module):
    """S4-based model for EEG Challenge 2025"""
    
    def __init__(
        self,
        n_chans: int = 129,
        n_times: int = 200,  # 2 seconds @ 100Hz
        d_model: int = 128,
        n_layers: int = 4,
        d_state: int = 32,
        dropout: float = 0.1,
        n_outputs: int = 1  # Response time regression
    ):
        super().__init__()
        
        self.n_chans = n_chans
        self.n_times = n_times
        self.d_model = d_model
        
        # Input projection: channels -> d_model
        self.input_projection = nn.Linear(n_chans, d_model)
        self.input_norm = nn.LayerNorm(d_model)
        
        # Positional encoding
        self.pos_encoding = self._create_positional_encoding(n_times, d_model)
        
        # S4 backbone
        self.s4_layers = nn.ModuleList([
            S4Layer(d_model, d_state, dropout) for _ in range(n_layers)
        ])
        
        self.layer_norms = nn.ModuleList([
            nn.LayerNorm(d_model) for _ in range(n_layers)
        ])
        
        # Global pooling and output head
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.output_head = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model // 2, n_outputs)
        )
        
    def _create_positional_encoding(self, seq_len: int, d_model: int) -> torch.Tensor:
        """Create sinusoidal positional encoding"""
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len).unsqueeze(1).float()
        
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * 
                           -(np.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        return pe.unsqueeze(0)  # (1, seq_len, d_model)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass
        Args:
            x: Input EEG tensor (batch_size, n_chans, n_times)
        Returns:
            Response time predictions (batch_size, 1)
        """
        # Transpose to (batch_size, n_times, n_chans) for temporal modeling
        x = x.transpose(1, 2)  # (batch, n_times, n_chans)
        
        # Project to d_model dimensions
        x = self.input_projection(x)  # (batch, n_times, d_model)
        x = self.input_norm(x)
        
        # Add positional encoding
        pos_enc = self.pos_encoding[:, :x.size(1), :].to(x.device)
        x = x + pos_enc
        
        # Apply S4 layers with residual connections
        for i, (s4_layer, norm) in enumerate(zip(self.s4_layers, self.layer_norms)):
            residual = x
            x = s4_layer(x)
            x = norm(x + residual)  # Residual connection
        
        # Global pooling: (batch, n_times, d_model) -> (batch, d_model)
        x = x.transpose(1, 2)  # (batch, d_model, n_times)
        x = self.global_pool(x).squeeze(-1)  # (batch, d_model)
        
        # Output prediction
        output = self.output_head(x)  # (batch, 1)
        
        return output.squeeze(-1)  # (batch,) for regression

## 2. Data Loading and Preprocessing

Load EEG data using the Challenge dataset format from EEGDash.

In [ ]:
# Import EEGDash and Braindecode for data handling
from pathlib import Path
from eegdash.dataset import EEGChallengeDataset
from braindecode.preprocessing import preprocess, Preprocessor, create_windows_from_events
from braindecode.datasets import BaseConcatDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

# Data directory
DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

print("Loading EEG Challenge Dataset...")
# Load the Contrast Change Detection task dataset
dataset_ccd = EEGChallengeDataset(
    task="contrastChangeDetection",
    release="R5", 
    cache_dir=DATA_DIR,
    mini=True  # Use mini dataset for faster testing
)

print(f"Dataset loaded: {len(dataset_ccd.datasets)} recordings")

In [ ]:
# Import utility functions from the starter kit
import numpy as np
import pandas as pd
import mne
from mne_bids import get_bids_path_from_fname

# Utility functions for trial extraction (from starter kit)
def build_trial_table(events_df: pd.DataFrame) -> pd.DataFrame:
    """Extract trial information with stimulus/response timing"""
    events_df = events_df.copy()
    events_df["onset"] = pd.to_numeric(events_df["onset"], errors="raise")
    events_df = events_df.sort_values("onset", kind="mergesort").reset_index(drop=True)

    trials = events_df[events_df["value"].eq("contrastTrial_start")].copy()
    stimuli = events_df[events_df["value"].isin(["left_target", "right_target"])].copy()
    responses = events_df[events_df["value"].isin(["left_buttonPress", "right_buttonPress"])].copy()

    trials = trials.reset_index(drop=True)
    trials["next_onset"] = trials["onset"].shift(-1)
    trials = trials.dropna(subset=["next_onset"]).reset_index(drop=True)

    rows = []
    for _, tr in trials.iterrows():
        start = float(tr["onset"])
        end   = float(tr["next_onset"])

        stim_block = stimuli[(stimuli["onset"] >= start) & (stimuli["onset"] < end)]
        stim_onset = np.nan if stim_block.empty else float(stim_block.iloc[0]["onset"])

        if not np.isnan(stim_onset):
            resp_block = responses[(responses["onset"] >= stim_onset) & (responses["onset"] < end)]
        else:
            resp_block = responses[(responses["onset"] >= start) & (responses["onset"] < end)]

        if resp_block.empty:
            resp_onset = np.nan
            resp_type  = None
            feedback   = None
        else:
            resp_onset = float(resp_block.iloc[0]["onset"])
            resp_type  = resp_block.iloc[0]["value"]
            feedback   = resp_block.iloc[0]["feedback"]

        rt_from_stim  = (resp_onset - stim_onset) if (not np.isnan(stim_onset) and not np.isnan(resp_onset)) else np.nan
        rt_from_trial = (resp_onset - start)       if not np.isnan(resp_onset) else np.nan

        correct = None
        if isinstance(feedback, str):
            if feedback == "smiley_face": correct = True
            elif feedback == "sad_face":  correct = False

        rows.append({
            "trial_start_onset": start,
            "trial_stop_onset": end,
            "stimulus_onset": stim_onset,
            "response_onset": resp_onset,
            "rt_from_stimulus": rt_from_stim,
            "rt_from_trialstart": rt_from_trial,
            "response_type": resp_type,
            "correct": correct,
        })

    return pd.DataFrame(rows)

def _to_float_or_none(x):
    return None if pd.isna(x) else float(x)

def _to_int_or_none(x):
    if pd.isna(x):
        return None
    if isinstance(x, (bool, np.bool_)):
        return int(bool(x))
    if isinstance(x, (int, np.integer)):
        return int(x)
    try:
        return int(x)
    except Exception:
        return None

def _to_str_or_none(x):
    return None if (x is None or (isinstance(x, float) and np.isnan(x))) else str(x)

def annotate_trials_with_target(raw, target_field="rt_from_stimulus", epoch_length=2.0,
                                require_stimulus=True, require_response=True):
    """Create trial annotations with response time targets"""
    fnames = raw.filenames
    assert len(fnames) == 1, "Expected a single filename"
    bids_path = get_bids_path_from_fname(fnames[0])
    events_file = bids_path.update(suffix="events", extension=".tsv").fpath

    events_df = (pd.read_csv(events_file, sep="\t")
                   .assign(onset=lambda d: pd.to_numeric(d["onset"], errors="raise"))
                   .sort_values("onset", kind="mergesort").reset_index(drop=True))

    trials = build_trial_table(events_df)

    if require_stimulus:
        trials = trials[trials["stimulus_onset"].notna()].copy()
    if require_response:
        trials = trials[trials["response_onset"].notna()].copy()

    if target_field not in trials.columns:
        raise KeyError(f"{target_field} not in computed trial table.")
    targets = trials[target_field].astype(float)

    onsets     = trials["trial_start_onset"].to_numpy(float)
    durations  = np.full(len(trials), float(epoch_length), dtype=float)
    descs      = ["contrast_trial_start"] * len(trials)

    extras = []
    for i, v in enumerate(targets):
        row = trials.iloc[i]

        extras.append({
            "target": _to_float_or_none(v),
            "rt_from_stimulus": _to_float_or_none(row["rt_from_stimulus"]),
            "rt_from_trialstart": _to_float_or_none(row["rt_from_trialstart"]),
            "stimulus_onset": _to_float_or_none(row["stimulus_onset"]),
            "response_onset": _to_float_or_none(row["response_onset"]),
            "correct": _to_int_or_none(row["correct"]),
            "response_type": _to_str_or_none(row["response_type"]),
        })

    new_ann = mne.Annotations(onset=onsets, duration=durations, description=descs,
                              orig_time=raw.info["meas_date"], extras=extras)
    raw.set_annotations(new_ann, verbose=False)
    return raw

def add_aux_anchors(raw, stim_desc="stimulus_anchor", resp_desc="response_anchor"):
    """Add stimulus and response anchor events"""
    ann = raw.annotations
    mask = (ann.description == "contrast_trial_start")
    if not np.any(mask):
        return raw

    stim_onsets, resp_onsets = [], []
    stim_extras, resp_extras = [], []

    for idx in np.where(mask)[0]:
        ex = ann.extras[idx] if ann.extras is not None else {}
        t0 = float(ann.onset[idx])

        stim_t = ex["stimulus_onset"]
        resp_t = ex["response_onset"]

        if stim_t is None or (isinstance(stim_t, float) and np.isnan(stim_t)):
            rtt = ex["rt_from_trialstart"]
            rts = ex["rt_from_stimulus"]
            if rtt is not None and rts is not None:
                stim_t = t0 + float(rtt) - float(rts)

        if resp_t is None or (isinstance(resp_t, float) and np.isnan(resp_t)):
            rtt = ex["rt_from_trialstart"]
            if rtt is not None:
                resp_t = t0 + float(rtt)

        if (stim_t is not None) and not (isinstance(stim_t, float) and np.isnan(stim_t)):
            stim_onsets.append(float(stim_t))
            stim_extras.append(dict(ex, anchor="stimulus"))
        if (resp_t is not None) and not (isinstance(resp_t, float) and np.isnan(resp_t)):
            resp_onsets.append(float(resp_t))
            resp_extras.append(dict(ex, anchor="response"))

    new_onsets = np.array(stim_onsets + resp_onsets, dtype=float)
    if len(new_onsets):
        aux = mne.Annotations(
            onset=new_onsets,
            duration=np.zeros_like(new_onsets, dtype=float),
            description=[stim_desc]*len(stim_onsets) + [resp_desc]*len(resp_onsets),
            orig_time=raw.info["meas_date"],
            extras=stim_extras + resp_extras,
        )
        raw.set_annotations(ann + aux, verbose=False)
    return raw

def keep_only_recordings_with(desc, concat_ds):
    """Keep only recordings that contain a specific event"""
    kept = []
    for ds in concat_ds.datasets:
        if np.any(ds.raw.annotations.description == desc):
            kept.append(ds)
        else:
            print(f"[warn] Recording {ds.raw.filenames[0]} does not contain event '{desc}'")
    return BaseConcatDataset(kept)

print("Utility functions defined successfully")

In [ ]:
# Create windowed data with stimulus-locked epochs
EPOCH_LEN_S = 2.0  # 2 second epochs
SFREQ = 100  # 100 Hz sampling rate

print("Processing trials and creating annotations...")

# Apply preprocessing to extract trials and create annotations
transformation_offline = [
    Preprocessor(
        annotate_trials_with_target,
        target_field="rt_from_stimulus", 
        epoch_length=EPOCH_LEN_S,
        require_stimulus=True, 
        require_response=True,
        apply_on_array=False,
    ),
    Preprocessor(add_aux_anchors, apply_on_array=False),
]

preprocess(dataset_ccd, transformation_offline, n_jobs=1)

# Use stimulus anchor for epoching
ANCHOR = "stimulus_anchor"
SHIFT_AFTER_STIM = 0.5  # 500ms shift after stimulus
WINDOW_LEN = 2.0        # 2 second window

# Keep only recordings with stimulus anchors
dataset = keep_only_recordings_with(ANCHOR, dataset_ccd)

print(f"Creating windowed epochs from {len(dataset.datasets)} recordings...")

# Create stimulus-locked windows
single_windows = create_windows_from_events(
    dataset,
    mapping={ANCHOR: 0},
    trial_start_offset_samples=int(SHIFT_AFTER_STIM * SFREQ),
    trial_stop_offset_samples=int((SHIFT_AFTER_STIM + WINDOW_LEN) * SFREQ),
    window_size_samples=int(EPOCH_LEN_S * SFREQ),  # 200 samples
    window_stride_samples=SFREQ,
    preload=True,
)

print(f"Created {len(single_windows)} windowed epochs")

In [ ]:
# Add target metadata to windows
def add_extras_columns(
    windows_concat_ds,
    original_concat_ds,
    desc="contrast_trial_start",
    keys=("target","rt_from_stimulus","rt_from_trialstart","stimulus_onset","response_onset","correct","response_type"),
):
    float_cols = {"target","rt_from_stimulus","rt_from_trialstart","stimulus_onset","response_onset"}

    for win_ds, base_ds in zip(windows_concat_ds.datasets, original_concat_ds.datasets):
        ann = base_ds.raw.annotations
        idx = np.where(ann.description == desc)[0]
        if idx.size == 0:
            continue

        per_trial = [
            {k: (ann.extras[i][k] if ann.extras is not None and k in ann.extras[i] else None) for k in keys}
            for i in idx
        ]

        md = win_ds.metadata.copy()
        first = (md["i_window_in_trial"].to_numpy() == 0)
        trial_ids = first.cumsum() - 1
        n_trials = trial_ids.max() + 1 if len(trial_ids) else 0
        assert n_trials == len(per_trial), f"Trial mismatch: {n_trials} vs {len(per_trial)}"

        for k in keys:
            vals = [per_trial[t][k] if t < len(per_trial) else None for t in trial_ids]
            if k == "correct":
                ser = pd.Series([None if v is None else int(bool(v)) for v in vals],
                                index=md.index, dtype="Int64")
            elif k in float_cols:
                ser = pd.Series([np.nan if v is None else float(v) for v in vals],
                                index=md.index, dtype="Float64")
            else:  # response_type
                ser = pd.Series(vals, index=md.index, dtype="string")

            md[k] = ser

        win_ds.metadata = md.reset_index(drop=True)
        if hasattr(win_ds, "y"):
            y_np = win_ds.metadata["target"].astype(float).to_numpy()
            win_ds.y = y_np[:, None]  # (N, 1)

    return windows_concat_ds

# Add response time targets to windowed data
single_windows = add_extras_columns(
    single_windows,
    dataset,
    desc=ANCHOR,
    keys=("target", "rt_from_stimulus", "rt_from_trialstart",
          "stimulus_onset", "response_onset", "correct", "response_type")
)

print("Metadata added to windowed epochs")

## 3. Train-Validation-Test Split

Split data at the subject level for proper generalization evaluation.

In [ ]:
# Get metadata and create subject-level splits
meta_information = single_windows.get_metadata()

print(f"Total windows: {len(meta_information)}")
print(f"Response time range: {meta_information['target'].min():.3f} - {meta_information['target'].max():.3f} seconds")
print(f"Subjects: {meta_information['subject'].unique()}")

# Subject-level train/validation/test split
valid_frac = 0.15
test_frac = 0.15
seed = 2025

subjects = meta_information["subject"].unique()
print(f"Total subjects: {len(subjects)}")

train_subj, valid_test_subject = train_test_split(
    subjects, test_size=(valid_frac + test_frac), random_state=check_random_state(seed), shuffle=True
)

valid_subj, test_subj = train_test_split(
    valid_test_subject, test_size=test_frac/(valid_frac + test_frac), 
    random_state=check_random_state(seed + 1), shuffle=True
)

# Create splits using braindecode functionality
subject_split = single_windows.split("subject")
train_set = []
valid_set = []
test_set = []

for s in subject_split:
    if s in train_subj:
        train_set.append(subject_split[s])
    elif s in valid_subj:
        valid_set.append(subject_split[s])
    elif s in test_subj:
        test_set.append(subject_split[s])

train_set = BaseConcatDataset(train_set)
valid_set = BaseConcatDataset(valid_set)
test_set = BaseConcatDataset(test_set)

print(f"\nDataset splits:")
print(f"Train: {len(train_set)} epochs from {len(train_subj)} subjects")
print(f"Valid: {len(valid_set)} epochs from {len(valid_subj)} subjects") 
print(f"Test: {len(test_set)} epochs from {len(test_subj)} subjects")

In [ ]:
# Create PyTorch DataLoaders
batch_size = 32
num_workers = 2

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

print(f"DataLoaders created:")
print(f"Train: {len(train_loader)} batches")
print(f"Valid: {len(valid_loader)} batches")
print(f"Test: {len(test_loader)} batches")

## 4. Model Training

Train the S4 model for response time prediction.

In [ ]:
# Initialize S4 model
model = EEGS4Model(
    n_chans=129,      # EEG channels
    n_times=200,      # Time points (2s @ 100Hz)
    d_model=128,      # Hidden dimension
    n_layers=4,       # S4 layers
    d_state=32,       # State dimension  
    dropout=0.1,      # Dropout rate
    n_outputs=1       # Response time regression
).to(device)

print(f"Model initialized with {sum(p.numel() for p in model.parameters()):,} parameters")

# Training configuration
lr = 1e-3
weight_decay = 1e-4
n_epochs = 50
patience = 10

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
criterion = nn.MSELoss()

print(f"Training configuration:")
print(f"Learning rate: {lr}")
print(f"Weight decay: {weight_decay}")
print(f"Epochs: {n_epochs}")
print(f"Patience: {patience}")

In [ ]:
# Training functions
from tqdm import tqdm
import copy

def train_epoch(model, train_loader, optimizer, criterion, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    n_batches = 0
    
    for batch_idx, batch in enumerate(tqdm(train_loader, desc="Training")):
        X, y = batch[0], batch[1]  # EEG data and response times
        X, y = X.to(device).float(), y.to(device).float()
        
        optimizer.zero_grad()
        predictions = model(X)
        loss = criterion(predictions, y.squeeze())
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        n_batches += 1
    
    return total_loss / n_batches

@torch.no_grad()
def validate_epoch(model, valid_loader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    total_loss = 0.0
    total_mae = 0.0
    n_batches = 0
    n_samples = 0
    
    for batch_idx, batch in enumerate(tqdm(valid_loader, desc="Validation")):
        X, y = batch[0], batch[1]
        X, y = X.to(device).float(), y.to(device).float()
        
        predictions = model(X)
        loss = criterion(predictions, y.squeeze())
        mae = F.l1_loss(predictions, y.squeeze())
        
        total_loss += loss.item()
        total_mae += mae.item() * X.size(0)
        n_batches += 1
        n_samples += X.size(0)
    
    avg_loss = total_loss / n_batches
    avg_mae = total_mae / n_samples
    rmse = np.sqrt(avg_loss)
    
    return avg_loss, avg_mae, rmse

print("Training functions defined")

In [ ]:
# Training loop with early stopping
print("Starting training...")

best_val_loss = float('inf')
best_model_state = None
patience_counter = 0
train_losses = []
val_losses = []
val_maes = []
val_rmses = []

for epoch in range(1, n_epochs + 1):
    print(f"\nEpoch {epoch}/{n_epochs}")
    print("-" * 50)
    
    # Training
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    train_losses.append(train_loss)
    
    # Validation  
    val_loss, val_mae, val_rmse = validate_epoch(model, valid_loader, criterion, device)
    val_losses.append(val_loss)
    val_maes.append(val_mae)
    val_rmses.append(val_rmse)
    
    # Learning rate scheduling
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    print(f"Train Loss: {train_loss:.6f}")
    print(f"Val Loss: {val_loss:.6f}, Val MAE: {val_mae:.6f}, Val RMSE: {val_rmse:.6f}")
    print(f"Learning Rate: {current_lr:.8f}")
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = copy.deepcopy(model.state_dict())
        patience_counter = 0
        print(f"New best validation loss: {best_val_loss:.6f}")
    else:
        patience_counter += 1
        print(f"No improvement for {patience_counter} epochs")
        
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch} epochs")
            break

# Load best model
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"\nLoaded best model with validation loss: {best_val_loss:.6f}")

print("\nTraining completed!")

## 5. Model Evaluation

Evaluate the trained S4 model on the test set.

In [ ]:
# Test evaluation
print("Evaluating on test set...")

@torch.no_grad()
def evaluate_test(model, test_loader, criterion, device):
    """Comprehensive test evaluation"""
    model.eval()
    
    all_predictions = []
    all_targets = []
    total_loss = 0.0
    n_batches = 0
    
    for batch in tqdm(test_loader, desc="Testing"):
        X, y = batch[0], batch[1]
        X, y = X.to(device).float(), y.to(device).float()
        
        predictions = model(X)
        loss = criterion(predictions, y.squeeze())
        
        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(y.squeeze().cpu().numpy())
        total_loss += loss.item()
        n_batches += 1
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    
    # Calculate metrics
    test_loss = total_loss / n_batches
    test_mae = np.mean(np.abs(all_predictions - all_targets))
    test_rmse = np.sqrt(np.mean((all_predictions - all_targets) ** 2))
    test_r2 = np.corrcoef(all_predictions, all_targets)[0, 1] ** 2
    
    return {
        'loss': test_loss,
        'mae': test_mae,
        'rmse': test_rmse,
        'r2': test_r2,
        'predictions': all_predictions,
        'targets': all_targets
    }

# Run test evaluation
test_results = evaluate_test(model, test_loader, criterion, device)

print(f"\n=== TEST RESULTS ===")
print(f"Test Loss: {test_results['loss']:.6f}")
print(f"Test MAE: {test_results['mae']:.6f} seconds")
print(f"Test RMSE: {test_results['rmse']:.6f} seconds")
print(f"Test R²: {test_results['r2']:.6f}")
print(f"Number of test samples: {len(test_results['predictions'])}")

In [ ]:
# Plot training curves and results
import matplotlib.pyplot as plt

# Training curves
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Loss curves
epochs_range = range(1, len(train_losses) + 1)
axes[0].plot(epochs_range, train_losses, 'b-', label='Train Loss')
axes[0].plot(epochs_range, val_losses, 'r-', label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('MSE Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True)

# Validation metrics
axes[1].plot(epochs_range, val_maes, 'g-', label='Val MAE')
axes[1].plot(epochs_range, val_rmses, 'purple', label='Val RMSE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Error (seconds)')
axes[1].set_title('Validation Metrics')
axes[1].legend()
axes[1].grid(True)

# Predictions vs targets scatter plot
axes[2].scatter(test_results['targets'], test_results['predictions'], alpha=0.6)
axes[2].plot([test_results['targets'].min(), test_results['targets'].max()], 
             [test_results['targets'].min(), test_results['targets'].max()], 
             'r--', lw=2)
axes[2].set_xlabel('True Response Time (s)')
axes[2].set_ylabel('Predicted Response Time (s)')
axes[2].set_title(f'Predictions vs Ground Truth\n(R² = {test_results["r2"]:.3f})')
axes[2].grid(True)

plt.tight_layout()
plt.savefig('eeg_s4_training_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("Training results plotted and saved")

## 6. Model Saving and Summary

Save the trained model and provide a summary of the MVP solution.

In [ ]:
# Save the trained model
model_save_path = 'eeg_s4_challenge_model.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'model_config': {
        'n_chans': 129,
        'n_times': 200,
        'd_model': 128,
        'n_layers': 4,
        'd_state': 32,
        'dropout': 0.1,
        'n_outputs': 1
    },
    'test_results': test_results,
    'training_config': {
        'lr': lr,
        'weight_decay': weight_decay,
        'batch_size': batch_size,
        'epochs_trained': len(train_losses)
    }
}, model_save_path)

print(f"Model saved to: {model_save_path}")

# Create a results summary
summary = f"""
=== EEG Challenge 2025 - S4 Model MVP Summary ===

MODEL ARCHITECTURE:
- S4-based temporal sequence model
- Input: 129 EEG channels × 200 time points (2s @ 100Hz)
- Hidden dimension: 128
- S4 layers: 4
- State dimension: 32
- Total parameters: {sum(p.numel() for p in model.parameters()):,}

TRAINING DATA:
- Total epochs: {len(single_windows)}
- Train: {len(train_set)} epochs from {len(train_subj)} subjects
- Validation: {len(valid_set)} epochs from {len(valid_subj)} subjects
- Test: {len(test_set)} epochs from {len(test_subj)} subjects

PERFORMANCE:
- Test MAE: {test_results['mae']:.6f} seconds
- Test RMSE: {test_results['rmse']:.6f} seconds
- Test R²: {test_results['r2']:.6f}
- Training epochs: {len(train_losses)}
- Best validation loss: {best_val_loss:.6f}

KEY FEATURES:
- Stimulus-locked epoching (+0.5s shift)
- Cross-subject generalization
- Response time regression
- Subject-level train/val/test splits
- Early stopping with patience={patience}

NEXT STEPS FOR IMPROVEMENT:
1. Implement full S4 block from eeg-pretraining repository
2. Add cross-paradigm transfer learning (SuS → CCD)
3. Incorporate domain adaptation layers
4. Add data augmentation techniques
5. Hyperparameter optimization
6. Ensemble methods
7. Use full dataset (not mini version)
"""

print(summary)

# Save summary to file
with open('eeg_s4_mvp_summary.txt', 'w') as f:
    f.write(summary)

print("\nSummary saved to: eeg_s4_mvp_summary.txt")

## 7. Challenge Submission Preparation

Prepare the model for submission to the EEG Challenge 2025.

In [ ]:
# Create submission-ready model class
class EEGChallengeSubmissionModel(nn.Module):
    """Submission-ready model for EEG Challenge 2025"""
    
    def __init__(self):
        super().__init__()
        
        # Initialize S4 model with the trained configuration
        self.s4_model = EEGS4Model(
            n_chans=129,
            n_times=200,
            d_model=128,
            n_layers=4,
            d_state=32,
            dropout=0.1,
            n_outputs=1
        )
        
    def forward(self, x):
        """Forward pass for submission
        Args:
            x: EEG tensor of shape (batch_size, n_chans=129, n_times=200)
        Returns:
            Response time predictions (batch_size,)
        """
        return self.s4_model(x)
    
    def predict(self, x):
        """Prediction method for challenge submission"""
        self.eval()
        with torch.no_grad():
            predictions = self.forward(x)
        return predictions

# Create submission model and load trained weights
submission_model = EEGChallengeSubmissionModel()
submission_model.s4_model.load_state_dict(model.state_dict())

print("Submission model created and weights loaded")

# Test the submission model format
dummy_input = torch.randn(4, 129, 200)  # Batch of 4 samples
test_output = submission_model.predict(dummy_input)
print(f"Submission model test - Input: {dummy_input.shape}, Output: {test_output.shape}")

# Save submission model
submission_path = 'eeg_challenge_2025_submission_s4.pth'
torch.save(submission_model.state_dict(), submission_path)
print(f"Submission model saved to: {submission_path}")

## Conclusion

This MVP notebook demonstrates a basic S4-based solution for the EEG Challenge 2025. The implementation includes:

✅ **Core Components**:
- S4 architecture for temporal EEG modeling
- Challenge-compliant data loading (129 channels, 200 time points)
- Stimulus-locked epoching with proper timing
- Response time regression
- Subject-level generalization splits

🚀 **Performance Achieved**:
- Baseline S4 model successfully trains and predicts response times
- Proper evaluation metrics (MAE, RMSE, R²)
- Submission-ready model format

📈 **Future Improvements**:
1. **Full S4 Implementation**: Replace simplified S4 with complete implementation from eeg-pretraining
2. **Cross-Paradigm Transfer**: Implement SuS → CCD transfer learning
3. **Domain Adaptation**: Add adversarial training for cross-task generalization
4. **Data Augmentation**: Time shifting, noise injection, channel dropout
5. **Architecture Enhancements**: Multi-head attention, deeper networks, ensemble methods
6. **Full Dataset**: Scale to complete R5 release instead of mini version

This serves as a solid foundation for building more sophisticated solutions to the EEG Foundation Challenge 2025!